In [ ]:
from pathlib import Path
from ebmdatalab import bq

from config import DATA_DIR

PROJECT_ID = "ebmdatalab"
CREDENTIALS_PATH = 'notebooks/cred/bq-service-account.json'


Fetch all trusts and whether they have A&E activity in last 6 months

In [15]:
all_ods_codes_query = """
WITH latest_six_months AS (
    SELECT DISTINCT period
    FROM `ebmdatalab.scmd_pipeline.org_ae_status`
    WHERE period >= DATE_SUB(DATE('2025-06-01'), INTERVAL 6 MONTH)
    ORDER BY period DESC
    LIMIT 6
),
ae_status_last_six_months AS (
    SELECT 
        ods_code,
        MAX(CAST(has_ae AS INT64)) AS has_ae_last_six_months
    FROM `ebmdatalab.scmd_pipeline.org_ae_status`
    WHERE period IN (SELECT period FROM latest_six_months)
    GROUP BY ods_code
)
SELECT DISTINCT 
    CASE
        WHEN ARRAY_LENGTH(ods.ultimate_successors) > 0 
        THEN ods.ultimate_successors[OFFSET(0)]
        ELSE ods.ods_code
    END AS ods_code,
    COALESCE(successor_org.ods_name, ods.ods_name) AS ods_name,
    CASE
        WHEN ae.ods_code IS NULL THEN FALSE
        WHEN ae.has_ae_last_six_months = 1 THEN TRUE
    ELSE FALSE
    END AS has_ae_last_six_months
FROM 
    `ebmdatalab.scmd_pipeline.ods` AS ods
LEFT JOIN 
    `ebmdatalab.scmd_pipeline.ods` AS successor_org
ON
    (CASE
        WHEN ARRAY_LENGTH(ods.ultimate_successors) > 0 
        THEN ods.ultimate_successors[OFFSET(0)]
        ELSE NULL
    END) = successor_org.ods_code
LEFT JOIN
    ae_status_last_six_months AS ae
ON 
    CASE
        WHEN ARRAY_LENGTH(ods.ultimate_successors) > 0 
        THEN ods.ultimate_successors[OFFSET(0)]
        ELSE ods.ods_code
    END = ae.ods_code
WHERE
    ARRAY_LENGTH(ods.ultimate_successors) > 0
    OR (
        (ods.operational_closed_date IS NULL OR ods.operational_closed_date >= '2019-01-01')
        AND (ods.legal_closed_date IS NULL OR ods.legal_closed_date >= '2019-01-01')
    )
ORDER BY has_ae_last_six_months DESC
"""

all_ods_codes = bq.cached_read(sql=all_ods_codes_query, csv_path=Path(DATA_DIR, "all_ods_codes.csv"), use_cache=False)
all_ods_codes = all_ods_codes.sort_values(["has_ae_last_six_months", "ods_name"])
all_ods_codes.to_csv(Path(DATA_DIR, "all_ods_codes.csv"), index=False)
all_ods_codes.head()

Downloading: 100%|█████████████████████████████████████████████████████████████|


,ods_code,ods_name,has_ae_last_six_months
202,RVN,Avon And Wiltshire Mental Health Partnership N...,False
147,RWX,Berkshire Healthcare NHS Foundation Trust,False
199,RXT,Birmingham And Solihull Mental Health NHS Foun...,False
197,RYW,Birmingham Community Healthcare NHS Foundation...,False
189,TAJ,Black Country Healthcare NHS Foundation Trust,False


Fetch apixaban and rivaroxaban primary care prescribing data. The secondary care data is manually fetched from OpenPrescribing Hospitals.

In [3]:
doac_query = """
SELECT CAST(np.month AS DATE) AS month, np.regional_team, vmp.nm, vpi.strnt_nmrtr_val, vpi.strnt_nmrtr_uom, vpi.strnt_dnmtr_val, vpi.strnt_dnmtr_uom, sum(items) AS sum_items, sum(quantity) AS sum_quantity,
CASE 
  WHEN np.bnf_code LIKE '0208020Z0%' THEN 10
  WHEN np.bnf_code LIKE '0208020Y0%' THEN 20
  ELSE 0
END AS DDD
FROM hscic.normalised_prescribing np
INNER JOIN ebmdatalab.hscic.practices p ON np.practice = p.code
INNER JOIN hscic.practice_statistics ps ON p.code = ps.practice AND np.month = ps.month
INNER JOIN dmd.vmp vmp ON CONCAT(SUBSTR(np.bnf_code,0,9),'AA', SUBSTR(np.bnf_code,-2), SUBSTR(np.bnf_code,-2)) = vmp.bnf_code -- joins prescribing data to vmp table using generic BNF code
INNER JOIN dmd.vpi vpi ON vmp.id = vpi.vmp
WHERE 
  (np.bnf_code LIKE '0208020Z0%'
  OR
  np.bnf_code LIKE '0208020Y0%')
  AND
  np.month >= '2019-01-01'
  AND
  p.setting = 4 -- to restrict to GP practice type
  AND
  ps.total_list_size > 0 -- exclude anything without a patient list - unlikely to be a valid GP
  AND
  np.regional_team IS NOT null
GROUP BY month, np.regional_team, DDD, vmp.nm, vpi.strnt_nmrtr_val, vpi.strnt_nmrtr_uom, vpi.strnt_dnmtr_val, vpi.strnt_dnmtr_uom
ORDER BY month, np.regional_team, vmp.nm ASC
"""

doacs = bq.cached_read(sql=doac_query, csv_path=Path(DATA_DIR, "doacs_pc.csv"), use_cache=False)


Downloading: 100%|█████████████████████████████████████████████████████████████|
